# 5003 project code

#### In this projec, we implemented pair programming through vscode live share. Thus, we tried the findspark to start our spark in vscode below. We shoud install findspark first.

%pip install findspark

In [93]:
%pip install findspark


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [94]:
import findspark

findspark.init(spark_home='/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2',
                python_path='/Users/zhanghousu/conda/bin/python')

from pyspark import SparkContext
sc = SparkContext()

23/04/24 22:28:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/24 22:28:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/24 22:28:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [95]:
# import packages

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [96]:
# reading the files in RDD
credits = sc.textFile('movie/credits.csv')
links = sc.textFile('movie/links.csv')
ratings = sc.textFile('movie/ratings.csv')
movies_metadata = sc.textFile('movie/movies_metadata.csv')

In [97]:
spark = SparkSession(sc)
cred_df = spark.read.csv(credits, header=True, inferSchema=True, escape='"')
links_df = spark.read.csv(links, header=True, inferSchema=True, escape='"')
ratings_df = spark.read.csv(ratings, header=True, inferSchema=True, escape='"')
movies_metadata_df = spark.read.csv(movies_metadata, header=True, inferSchema=True, escape='"')


23/04/24 22:28:44 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


23/04/24 22:28:52 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 10): Attempting to kill Python Worker


23/04/24 22:29:05 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 6 (TID 33): Attempting to kill Python Worker


## cleanning credit dataset
#### calculating null data

In [98]:
# calucate the null values in each column
null_rows = cred_df.select(*[(sum(when(isnull(c), 1).otherwise(0))).alias(c) for c in cred_df.columns])
null_rows.show()

+----+----+---+
|cast|crew| id|
+----+----+---+
|   0|   0|  0|
+----+----+---+



In [99]:
# drop the rows with null values
cred_df_n = cred_df.dropna()
null_rows_check = cred_df_n.select(*[(sum(when(isnull(c), 1).otherwise(0))).alias(c) for c in cred_df_n.columns])
null_rows_check.show()

+----+----+---+
|cast|crew| id|
+----+----+---+
|   0|   0|  0|
+----+----+---+



## cleanning metadata dataset
#### removing useless features, calculating null data, data StandardScaler

In [100]:
# drop useless columns
mv_meta_df_dp = movies_metadata_df.drop('adult', 'belongs_to_collection', 'homepage', 'imdb_id', 
                                        'original_title', 'overview', 'poster_path', 'release_date', 
                                        'spoken_languages', 'title', 'tagline', 'video')

# show the size of the mv_meta_df_dp
mv_meta_df_dp.count()



45572

In [101]:
# delete the rows with status not equal to "Released"
mv_meta_df_dp = mv_meta_df_dp.filter(col("status") == "Released")
mv_meta_df_dp.count()

44941

In [102]:
# remove status column
mv_meta_df_dp = mv_meta_df_dp.drop('status')
mv_meta_df_dp.show(5)

23/04/24 22:29:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 20 (TID 56): Attempting to kill Python Worker
+--------+--------------------+-----+-----------------+----------+--------------------+--------------------+---------+-------+------------+----------+
|  budget|              genres|   id|original_language|popularity|production_companies|production_countries|  revenue|runtime|vote_average|vote_count|
+--------+--------------------+-----+-----------------+----------+--------------------+--------------------+---------+-------+------------+----------+
|30000000|[{'id': 16, 'name...|  862|               en| 21.946943|[{'name': 'Pixar ...|[{'iso_3166_1': '...|373554033|   81.0|         7.7|      5415|
|65000000|[{'id': 12, 'name...| 8844|               en| 17.015539|[{'name': 'TriSta...|[{'iso_3166_1': '...|262797249|  104.0|         6.9|      2413|
|       0|[{'id': 10749, 'n...|15602|               en|   11.7129|[{'name': 'Warner...|[{'iso_3166_1': '...| 

In [103]:
# remove the rows with budget and revenue equal to 0
mv_meta_df_dp.filter((col("budget") == 0) & (col("revenue") == 0)).count()

34065

In [104]:
mv_meta_df_dp = mv_meta_df_dp.drop('budget', 'revenue')
mv_meta_df_dp.show(5)

23/04/24 22:29:18 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 24 (TID 60): Attempting to kill Python Worker
+--------------------+-----+-----------------+----------+--------------------+--------------------+-------+------------+----------+
|              genres|   id|original_language|popularity|production_companies|production_countries|runtime|vote_average|vote_count|
+--------------------+-----+-----------------+----------+--------------------+--------------------+-------+------------+----------+
|[{'id': 16, 'name...|  862|               en| 21.946943|[{'name': 'Pixar ...|[{'iso_3166_1': '...|   81.0|         7.7|      5415|
|[{'id': 12, 'name...| 8844|               en| 17.015539|[{'name': 'TriSta...|[{'iso_3166_1': '...|  104.0|         6.9|      2413|
|[{'id': 10749, 'n...|15602|               en|   11.7129|[{'name': 'Warner...|[{'iso_3166_1': '...|  101.0|         6.5|        92|
|[{'id': 35, 'name...|31357|               en|  3.859495|[{'name': 'Twen

In [105]:
mv_meta_df_dp.printSchema()

root
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)



In [106]:
from pyspark.sql.types import DoubleType

# create DoubleType list
columns_to_convert = ["popularity", "runtime", "vote_average", "vote_count"]

# convert the columns to DoubleType
for column in columns_to_convert:
    mv_meta_df_dp = mv_meta_df_dp.withColumn(column, mv_meta_df_dp[column].cast(DoubleType()))

# check the schema
mv_meta_df_dp.printSchema()

root
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- runtime: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: double (nullable = true)



In [107]:
from pyspark.sql.functions import mean, stddev, col

# create a list of columns to normalize
for column in columns_to_convert:
    # calculate the mean and standard deviation
    mean_value, stddev_value = mv_meta_df_dp.select(mean(col(column)), stddev(col(column))).first()

    # normalize the column
    mv_meta_df_dp = mv_meta_df_dp.withColumn(column, (col(column) - mean_value) / stddev_value)

mv_meta_df_dp.show(5)


23/04/24 22:29:25 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 37 (TID 73): Attempting to kill Python Worker
+--------------------+-----+-----------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+
|              genres|   id|original_language|         popularity|production_companies|production_countries|            runtime|       vote_average|          vote_count|
+--------------------+-----+-----------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+
|[{'id': 16, 'name...|  862|               en| 3.1532246689818866|[{'name': 'Pixar ...|[{'iso_3166_1': '...|-0.3460900547748043| 1.0839043917589501|  10.736622418900206|
|[{'id': 12, 'name...| 8844|               en| 2.3350472627476364|[{'name': 'TriSta...|[{'iso_3166_1': '...|  0.253392902201182| 0.6661104271252892|   4.659792962045657|
|[{'id': 10749

In [108]:
import ast
# create a new column named "profit"
def extract_first_genre(genres_str):
    try:
        genres_list = ast.literal_eval(genres_str)
    except (ValueError, SyntaxError):
        return 'no genre'
    
    if len(genres_str) <= 2:
        return 'no genre'
    else:
        genres_list = ast.literal_eval(genres_str)
        return genres_list[0]['name']


# create a user defined function
first_genre_udf = udf(extract_first_genre, StringType())

In [109]:
# apply the udf to the dataframe
mv_meta_df_oh = mv_meta_df_dp.withColumn("first_genre", first_genre_udf(col("genres")))
mv_meta_df_oh.show(5)

23/04/24 22:29:29 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 38 (TID 74): Attempting to kill Python Worker
+--------------------+-----+-----------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-----------+
|              genres|   id|original_language|         popularity|production_companies|production_countries|            runtime|       vote_average|          vote_count|first_genre|
+--------------------+-----+-----------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-----------+
|[{'id': 16, 'name...|  862|               en| 3.1532246689818866|[{'name': 'Pixar ...|[{'iso_3166_1': '...|-0.3460900547748043| 1.0839043917589501|  10.736622418900206|  Animation|
|[{'id': 12, 'name...| 8844|               en| 2.3350472627476364|[{'name': 'TriSta...|[{'iso_3166_1': '...|  0.253392902201182| 0.66611

In [110]:
mv_meta_df_oh = mv_meta_df_oh.drop('genres')
mv_meta_df_oh.printSchema()


root
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- runtime: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: double (nullable = true)
 |-- first_genre: string (nullable = true)



In [111]:
mv_meta_df_oh.select("first_genre").distinct().count()

21

In [112]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType

# create a StringIndexer and OneHotEncoder
indexer = StringIndexer(inputCol="first_genre", outputCol="first_genre_index")
encoder = OneHotEncoder(inputCol="first_genre_index", outputCol="first_genre_onehot", dropLast=False)

# create a pipeline
pipeline = Pipeline(stages=[indexer, encoder])

# fit the pipeline to the dataframe
model = pipeline.fit(mv_meta_df_oh)
mv_meta_df_oh_encoded = model.transform(mv_meta_df_oh)


In [113]:

# create a function to convert a vector to an array
def vector_to_array(vec):
    if isinstance(vec, SparseVector):
        vec = DenseVector(vec)
    return vec.toArray().tolist()

vector_to_array_udf = udf(vector_to_array, ArrayType(DoubleType()))

# apply the function to the dataframe
mv_meta_df_oh_encoded = mv_meta_df_oh_encoded.withColumn("first_genre_onehot_array", vector_to_array_udf(col("first_genre_onehot")))



In [114]:
# get the number of categories
num_categories = len(model.stages[0].labels)
for i in range(num_categories):
    mv_meta_df_oh_encoded = mv_meta_df_oh_encoded.withColumn(
        f"FG_onehot_{i}",
        mv_meta_df_oh_encoded["first_genre_onehot_array"].getItem(i)
    )

# drop the original columns
mv_meta_df_oh_encoded = mv_meta_df_oh_encoded.drop("first_genre_onehot_array", "first_genre_onehot", 
                                                   "first_genre", "first_genre_index")
mv_meta_df_oh_encoded.show(5)

+-----+-----------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   id|original_language|         popularity|production_companies|production_countries|            runtime|       vote_average|          vote_count|FG_onehot_0|FG_onehot_1|FG_onehot_2|FG_onehot_3|FG_onehot_4|FG_onehot_5|FG_onehot_6|FG_onehot_7|FG_onehot_8|FG_onehot_9|FG_onehot_10|FG_onehot_11|FG_onehot_12|FG_onehot_13|FG_onehot_14|FG_onehot_15|FG_onehot_16|FG_onehot_17|FG_onehot_18|FG_onehot_19|FG_onehot_20|
+-----+-----------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-----------+-----------+----

In [115]:
mv_meta_df_oh_encoded.count()

44941

In [116]:
# create a new column named production_companies
def extract_first_prod(prod_str):
    try:
        prod_list = ast.literal_eval(prod_str)
    except (ValueError, SyntaxError):
        return 'no production company'

    if len(prod_str) <= 2:
        return 'no production company'
    else:
        prod_list = ast.literal_eval(prod_str)
        return prod_list[0]['name']


# create a user defined function
first_prod_udf = udf(extract_first_prod, StringType())

In [117]:
# apply the udf to the dataframe
mv_meta_df_prod = mv_meta_df_dp.withColumn("first_production", first_prod_udf(col("production_companies")))
mv_meta_df_prod.select("first_production").distinct().count()

10543

In [118]:
# too many categories of the production
mv_meta_df_oh_encoded = mv_meta_df_oh_encoded.drop('production_companies', 'original_language', 
                                   'spoken_languages', 'production_companies',
                                   'production_countries')
mv_meta_df_oh_encoded.show(5)

+-----+-------------------+-------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   id|         popularity|            runtime|       vote_average|          vote_count|FG_onehot_0|FG_onehot_1|FG_onehot_2|FG_onehot_3|FG_onehot_4|FG_onehot_5|FG_onehot_6|FG_onehot_7|FG_onehot_8|FG_onehot_9|FG_onehot_10|FG_onehot_11|FG_onehot_12|FG_onehot_13|FG_onehot_14|FG_onehot_15|FG_onehot_16|FG_onehot_17|FG_onehot_18|FG_onehot_19|FG_onehot_20|
+-----+-------------------+-------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+----------

## cleanning the credits dataset
#### obtain the first three casts and calculate a new avg rating table for the casts

In [119]:
from pyspark.sql.functions import from_json, col, explode,\
    get_json_object, regexp_replace, array
from pyspark.sql.types import StructType, StructField,\
    StringType, IntegerType, ArrayType, MapType, DoubleType

# read the data
cred_df = spark.read.csv('movie/credits.csv', header=True, escape='"')
movi_df = spark.read.csv('movie/movies_metadata.csv', header=True, escape='"')

# remove the rows with budget and revenue equal to 0
json_schema = ArrayType(StructType([
    StructField("id", IntegerType(), True),
]))

# remove the rows with budget and revenue equal to 0
cred_df = cred_df.withColumn('cast',regexp_replace("cast", 'None', "'None'"))
cred_df = cred_df.withColumn("cast_array", from_json(col("cast"),json_schema))
cred_df = cred_df.withColumn("cast_array",\
                             array(col('cast_array')[0],\
                                   col('cast_array')[1],\
                                   col('cast_array')[2]))

cast_df = cred_df.select(col("id").alias("movieId"), explode(col("cast_array"))\
                         .alias("cast_item"))
cast_name_df = cast_df.withColumn("castId", col("cast_item")["id"])\
                    .drop('cast_item')

cast_name_df.show(5)


+-------+------+
|movieId|castId|
+-------+------+
|    862|    31|
|    862| 12898|
|    862|  7167|
|   8844|  2157|
|   8844|  8537|
+-------+------+
only showing top 5 rows



In [120]:
# remove the rows with budget and revenue equal to 0
ratings_df = spark.read.csv('movie/ratings.csv', header=True, escape='"')
ratings_df = ratings_df.withColumn('rating', ratings_df['rating']\
                                   .cast(DoubleType()))
ratings_df = ratings_df.groupBy("movieId").avg("rating")\
                        .withColumnRenamed("avg(rating)", "rating")

cast_name_with_rating = cast_name_df\
                        .join(ratings_df,\
                        cast_name_df.movieId == ratings_df.movieId,"inner")\
                        .groupBy("castId")\
                        .avg("rating")\
                        .withColumnRenamed("avg(rating)", "rating")
cast_name_with_rating.show(5)

+-------+------------------+
| castId|            rating|
+-------+------------------+
|1362204|             3.375|
|  18979|3.7976480836236934|
|   1959| 3.250567650909128|
|  69637|3.7530362663626198|
| 107536|2.9383561643835616|
+-------+------------------+
only showing top 5 rows



In [121]:
# join the two dataframes
joined_df = cast_name_df.join(cast_name_with_rating, on="castId", how="inner")

# calculate the average rating for each movie
mv_cast_avg_rating = joined_df.groupBy("movieId").agg(avg("rating").alias("avg_cast_rating"))

# join the two dataframes
mv_cast_avg_rating.show(5)

+-------+------------------+
|movieId|   avg_cast_rating|
+-------+------------------+
| 141418|             3.375|
|  55321| 3.203446833591777|
|  47880|3.0102890144060197|
|  47940|3.3937500000000003|
| 356054| 3.208588957055215|
+-------+------------------+
only showing top 5 rows



In [122]:
mv_cast_avg_rating.count()

33125

## Join the datasets

In [123]:
# rename the it to movieId
mv_meta_df_oh_encoded = mv_meta_df_oh_encoded.withColumnRenamed("id", "movieId")
mv_meta_df_oh_encoded.drop("id")
mv_meta_df_oh_encoded.show(5)


+-------+-------------------+-------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|movieId|         popularity|            runtime|       vote_average|          vote_count|FG_onehot_0|FG_onehot_1|FG_onehot_2|FG_onehot_3|FG_onehot_4|FG_onehot_5|FG_onehot_6|FG_onehot_7|FG_onehot_8|FG_onehot_9|FG_onehot_10|FG_onehot_11|FG_onehot_12|FG_onehot_13|FG_onehot_14|FG_onehot_15|FG_onehot_16|FG_onehot_17|FG_onehot_18|FG_onehot_19|FG_onehot_20|
+-------+-------------------+-------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+----

In [124]:
# join the two dataframes
main = mv_meta_df_oh_encoded.join(mv_cast_avg_rating, on="movieId", how="inner")
main.show(5)

+-------+--------------------+--------------------+------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+
|movieId|          popularity|             runtime|      vote_average|          vote_count|FG_onehot_0|FG_onehot_1|FG_onehot_2|FG_onehot_3|FG_onehot_4|FG_onehot_5|FG_onehot_6|FG_onehot_7|FG_onehot_8|FG_onehot_9|FG_onehot_10|FG_onehot_11|FG_onehot_12|FG_onehot_13|FG_onehot_14|FG_onehot_15|FG_onehot_16|FG_onehot_17|FG_onehot_18|FG_onehot_19|FG_onehot_20|   avg_cast_rating|
+-------+--------------------+--------------------+------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+--

In [125]:
ratings_df.count()

45115

In [126]:
main.count()

32828

In [127]:
# join the main and ratings dataframe
main = main.join(ratings_df, on="movieId", how="inner")

In [128]:
main = main.drop("movieId")
main.count()

7231

In [129]:
main.show(5)

+--------------------+--------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+------------------+
|          popularity|             runtime|       vote_average|          vote_count|FG_onehot_0|FG_onehot_1|FG_onehot_2|FG_onehot_3|FG_onehot_4|FG_onehot_5|FG_onehot_6|FG_onehot_7|FG_onehot_8|FG_onehot_9|FG_onehot_10|FG_onehot_11|FG_onehot_12|FG_onehot_13|FG_onehot_14|FG_onehot_15|FG_onehot_16|FG_onehot_17|FG_onehot_18|FG_onehot_19|FG_onehot_20|   avg_cast_rating|            rating|
+--------------------+--------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------

## Machine Learning

## Machine Learning

In [130]:
# import the necessary libraries
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import when, col

In [131]:


threshold = 3.0  # Replace with the threshold you want to use for classification
main = main.withColumn("label", when(col("rating") > threshold, 1).otherwise(0))


In [132]:
final_df = main.drop("rating")
final_df.show(5)

+--------------------+--------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+-----+
|          popularity|             runtime|       vote_average|          vote_count|FG_onehot_0|FG_onehot_1|FG_onehot_2|FG_onehot_3|FG_onehot_4|FG_onehot_5|FG_onehot_6|FG_onehot_7|FG_onehot_8|FG_onehot_9|FG_onehot_10|FG_onehot_11|FG_onehot_12|FG_onehot_13|FG_onehot_14|FG_onehot_15|FG_onehot_16|FG_onehot_17|FG_onehot_18|FG_onehot_19|FG_onehot_20|   avg_cast_rating|label|
+--------------------+--------------------+-------------------+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+-----------

In [133]:
# Assuming 'label' is your label column
label_column = 'label'

# Get a list of all columns
all_columns = final_df.columns

# Remove the label column from the list of feature columns
feature_columns = [column for column in all_columns if column != label_column]
final_df = final_df.na.drop(subset=feature_columns)


# Now feature_columns contains the names of all feature columns in your DataFrame


In [134]:
# Prepare for the pipeline
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline_lr = Pipeline(stages=[assembler, lr])

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()



In [136]:
# Create 5-fold CrossValidator
crossval_lr = CrossValidator(estimator=pipeline_lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  

train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)
cv_model_lr = crossval_lr.fit(train_data)




23/04/24 22:44:53 WARN CacheManager: Asked to cache already cached data.
23/04/24 22:44:53 WARN CacheManager: Asked to cache already cached data.


In [139]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_f1 = MulticlassClassificationEvaluator(metricName='f1')
evaluator_accuracy = MulticlassClassificationEvaluator(metricName='accuracy')
evaluator_recall = MulticlassClassificationEvaluator(metricName='weightedRecall')


In [140]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
predictions_lr = cv_model_lr.transform(test_data)
accuracy_lr = evaluator_accuracy.evaluate(predictions_lr)
f1_score_lr = evaluator_f1.evaluate(predictions_lr)
recall_lr = evaluator_recall.evaluate(predictions_lr)
print("Logistic Regression test set accuracy: ", accuracy_lr)
print("Logistic Regression test set F1 score: ", f1_score_lr)
print("Logistic Regression test set recall: ", recall_lr)


Logistic Regression test set accuracy:  0.6267247639796659
Logistic Regression test set F1 score:  0.4829138136736176
Logistic Regression test set recall:  0.6267247639796659


In [141]:
from pyspark.ml.classification import LinearSVC

# Prepare for the pipeline
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
svm = LinearSVC(maxIter=10)
pipeline_svm = Pipeline(stages=[assembler, svm])

paramGrid_svm = ParamGridBuilder() \
    .addGrid(svm.regParam, [0.1, 0.01]) \
    .addGrid(svm.fitIntercept, [False, True]) \
    .build()

# Create 5-fold CrossValidator
crossval_svm = CrossValidator(estimator=pipeline_svm,
                          estimatorParamMaps=paramGrid_svm,
                          evaluator=evaluator,
                          numFolds=5)  

cv_model_svm = crossval_svm.fit(train_data)

# Make predictions on test documents. cvModel uses the best model found.
predictions_svm = cv_model_svm.transform(test_data)
accuracy_svm = evaluator_accuracy.evaluate(predictions_svm)
f1_score_svm = evaluator_f1.evaluate(predictions_svm)
recall_svm = evaluator_recall.evaluate(predictions_svm)
print("Support Vector Machine test set accuracy: ", accuracy_svm)
print("Support Vector Machine test set F1 score: ", f1_score_svm)
print("Support Vector Machine test set recall: ", recall_svm)



Support Vector Machine test set accuracy:  0.869281045751634
Support Vector Machine test set F1 score:  0.8678482291075018
Support Vector Machine test set recall:  0.869281045751634


In [142]:
from pyspark.ml.classification import DecisionTreeClassifier

# Prepare for the pipeline
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
dt = DecisionTreeClassifier()
pipeline_dt = Pipeline(stages=[assembler, dt])

paramGrid_dt = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [3, 4, 5]) \
    .addGrid(dt.impurity, ["gini", "entropy"]) \
    .build()

# Create 5-fold CrossValidator
crossval_dt = CrossValidator(estimator=pipeline_dt,
                          estimatorParamMaps=paramGrid_dt,
                          evaluator=evaluator,
                          numFolds=5)  

cv_model_dt = crossval_dt.fit(train_data)

# Make predictions on test documents. cvModel uses the best model found.
predictions_dt = cv_model_dt.transform(test_data)
accuracy_dt = evaluator_accuracy.evaluate(predictions_dt)
f1_score_dt = evaluator_f1.evaluate(predictions_dt)
recall_dt = evaluator_recall.evaluate(predictions_dt)
print("Decision Tree test set accuracy: ", accuracy_dt)
print("Decision Tree test set F1 score: ", f1_score_dt)
print("Decision Tree test set recall: ", recall_dt)



Decision Tree test set accuracy:  0.8707334785766159
Decision Tree test set F1 score:  0.8706304459671901
Decision Tree test set recall:  0.8707334785766159


In [ ]:
# # stop the spark session
# spark.stop()